# The full pipeline for advanced lane recognition
## by Alok Rao

In [28]:
import numpy as np
import os
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob as glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## Parameter Class to load all parameters

In [29]:
class Parameters():
    def BinaryImageThreshold(self, sobelThreshold, sChannelThreshold, redChannelThreshold, sobelKernelSize):
        self.sobelThreshold = sobelThreshold
        self.sChannelThreshold = sChannelThreshold
        self.redChannelThreshold = redChannelThreshold
        self.sobelKernelSize = sobelKernelSize
        
    def FindWindowLanesParameters(self, nWindows, minPix):
        self.nWindows = nWindows
        self.minPix = minPix

## Camera calibration

In [30]:
def GetCameraCorrection(imagePath):
    
    objPoints = []
    imgPoints = []
    objP = np.zeros((9*6,3),np.float32)
    objP[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    #first find thee corners
    for filename in imagePath:
        img=mpimg.imread(filename)
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
        if ret == True:
            objPoints.append(objP)
            imgPoints.append(corners)
    
    # Returns camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objPoints, imgPoints, 
                                                       gray.shape[::-1], None, 
                                                       None)
    
    return mtx, dist

In [31]:
imageLocation = glob.glob('camera_cal/*.jpg')

# Calibrate camera and return calibration data
mtx, dist = GetCameraCorrection(imageLocation)

## Filter image to get rough lanes

In [32]:
def GetBinaryImage(image, mtx, dst):
    undistortedImage = cv2.undistort(image, mtx, dst, None, mtx)
    sobelThreshold = parameters.sobelThreshold
    sChannelThreshold = parameters.sChannelThreshold
    redChannelThreshold = parameters.redChannelThreshold
    sobelKernelSize = parameters.sobelKernelSize
    redImage = undistortedImage[:,:,0]
    hsvImage = cv2.cvtColor(undistortedImage, cv2.COLOR_RGB2HSV).astype(np.float)
    # Convert to HLS colorspace
    hlsImage = cv2.cvtColor(undistortedImage, cv2.COLOR_RGB2HLS).astype(np.float)
    sImage = hlsImage[:,:,2]
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    vImage = hsvImage[:,:,2]
    
    
    #Applying Sobel Filter, creaating Binary image
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobelKernelSize) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sxbinary = np.zeros_like(scaled_sobelx)
    sxbinary[(scaled_sobelx >= sobelThreshold[0]) & (scaled_sobelx <= sobelThreshold[1])] = 1
   
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobelKernelSize) # Take the derivative in x
    abs_sobely = np.absolute(sobely) # Absolute y derivative to accentuate lines away from horizontal
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= sobelThreshold[0]) & (scaled_sobely <= sobelThreshold[1])] = 1
    #Applying thresholding to S channel
    sBinary = np.zeros_like(sImage)
    sBinary[(sImage >= sChannelThreshold[0]) & (sImage <= sChannelThreshold[1])] = 1
    
    #Applying thresold Red channel
    rBinary = np.zeros_like(redImage)
    rBinary[(redImage>=redChannelThreshold[0]) & (redImage<=redChannelThreshold[1])] = 1
    
    #Applying thresold Red channel
    vBinary = np.zeros_like(vImage)
    vBinary[(vImage>=230) & (vImage<=255)] = 1
    
    #Stacking for debugging
    #combinedBinary = np.dstack(( np.zeros_like(sxbinary), sxbinary, sBinary)) * 255
    
    # Combine the  binary thresholds
    combinedBinary = np.zeros_like(sxbinary)
    combinedBinary[(vBinary == 1) | (sxbinary == 1) | (rBinary == 1)] = 1
    #combinedBinary[(vBinary == 1)] = 1
    
    return combinedBinary

## Convert image to to down perspective

In [34]:
def PerspectiveTransform(image, mtx, dst):
    #Perspective transform to obtain a top down image
    
    #First obtain binary image
    binaryImage = GetBinaryImage(image, mtx, dst)
    
    #get image size
    imageSize = (binaryImage.shape[1], binaryImage.shape[0])
    
    #get 4 reference points
    source = np.float32([[585,455],[705,455],[1130,720],[190,720]])
    
    #4 points from the image
    offset = 200 # offset for dst points
    dst = np.float32([
    [offset, 0],
    [imageSize[0]-offset, 0],
    [imageSize[0]-offset, imageSize[1]], 
    [offset, imageSize[1]]
])
    
    # Use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(source, dst)
    
    # Use cv2.warpPerspective() to warp the image to a top-down view
    topDown = cv2.warpPerspective(binaryImage, M, imageSize)

    return topDown, M

## Find the lane lines on the perspective image

In [35]:
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

def findLines(image, nwindows=9, margin=110, minpix=50):
    """
    Find the polynomial representation of the lines in the `image` using:
    - `nwindows` as the number of windows.
    - `margin` as the windows margin.
    - `minpix` as minimum number of pixes found to recenter the window.
    - `ym_per_pix` meters per pixel on Y.
    - `xm_per_pix` meters per pixels on X.
    
    Returns (left_fit, right_fit, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy)
    """    
    # Make a binary and transform image
    binary_warped, M = PerspectiveTransform(image, mtx, dist)
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit a second order polynomial to each
    left_fit_m = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_m = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    return (left_fit, right_fit, left_fit_m, right_fit_m, left_lane_inds, right_lane_inds, out_img, nonzerox, nonzeroy, M)


In [40]:

def calculateCurvature(yRange, left_fit_cr):
    """
    Returns the curvature of the polynomial `fit` on the y range `yRange`.
    """
    
    return ((1 + (2*left_fit_cr[0]*yRange*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])

def drawLine(img, left_fit, right_fit, M):
    """
    Draw the lane lines on the image `img` using the poly `left_fit` and `right_fit`.
    """
    yMax = img.shape[0]
    ploty = np.linspace(0, yMax - 1, yMax)
    color_warp = np.zeros_like(img).astype(np.uint8)
    
    # Calculate points.
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, np.linalg.inv(M), (img.shape[1], img.shape[0])) 
    return cv2.addWeighted(img, 1, newwarp, 0.3, 0)

In [41]:
from moviepy.editor import VideoFileClip

class Lane():
    def __init__(self):
        self.left_fit = None
        self.right_fit = None
        self.left_fit_m = None
        self.right_fit_m = None
        self.leftCurvature = None
        self.rightCurvature = None

def calculateLanes(img):
    """
    Calculates the lane on image `img`.
    """
    left_fit, right_fit, left_fit_m, right_fit_m, _, _, _, _, _,M = findLines(img)
    # Calculate curvature
    yRange = 719
    leftCurvature = calculateCurvature(yRange, left_fit_m) 
    rightCurvature = calculateCurvature(yRange, right_fit_m)
    
    # Calculate vehicle center
    xMax = img.shape[1]*xm_per_pix
    yMax = img.shape[0]*ym_per_pix
    vehicleCenter = xMax / 2
    lineLeft = left_fit_m[0]*yMax**2 + left_fit_m[1]*yMax + left_fit_m[2]
    lineRight = right_fit_m[0]*yMax**2 + right_fit_m[1]*yMax + right_fit_m[2]
    lineMiddle = lineLeft + (lineRight - lineLeft)/2
    diffFromVehicle = lineMiddle - vehicleCenter
    
    return (left_fit, right_fit, left_fit_m, right_fit_m, leftCurvature, rightCurvature, diffFromVehicle, M)

def displayLanes(img, left_fit, right_fit, left_fit_m, right_fit_m, leftCurvature, rightCurvature, diffFromVehicle,M):
    """
    Display the lanes information on the image.
    """
    output = drawLine(img, left_fit, right_fit,M)
    
    if diffFromVehicle > 0:
        message = '{:.2f} m right'.format(diffFromVehicle)
    else:
        message = '{:.2f} m left'.format(-diffFromVehicle)
    
    # Draw info
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    cv2.putText(output, 'Left curvature: {:.0f} m'.format(leftCurvature), (50, 50), font, 1, fontColor, 2)
    cv2.putText(output, 'Right curvature: {:.0f} m'.format(rightCurvature), (50, 120), font, 1, fontColor, 2)
    cv2.putText(output, 'Vehicle is {} of center'.format(message), (50, 190), font, 1, fontColor, 2)
    return output
    
def videoPipeline(inputVideo, outputVideo):
    """
    Process the `inputVideo` frame by frame to find the lane lines, draw curvarute and vehicle position information and
    generate `outputVideo`
    """
    myclip = VideoFileClip(inputVideo)
    parameters = Parameters()
    parameters.BinaryImageThreshold((70,160), (110,140), (215,255), 3)
    parameters.FindWindowLanesParameters(20,50)
    leftLane = Lane()
    rightLane = Lane()
    
    def processImage(img):
        left_fit, right_fit, left_fit_m, right_fit_m, leftCurvature, rightCurvature, diffFromVehicle,M = calculateLanes(img)
        if leftCurvature > 10000:
            left_fit = leftLane.left_fit
            left_fit_m = leftLane.left_fit_m
            leftCurvature = leftLane.leftCurvature
        else:
            leftLane.left_fit = left_fit
            leftLane.left_fit_m = left_fit_m
            leftLane.leftCurvature = leftCurvature
        
        if rightCurvature > 10000:
            right_fit = rightLane.right_fit
            right_fit_m = rightLane.right_fit_m
            rightCurvature = rightLane.rightCurvature
        else:
            rightLane.right_fit = right_fit
            rightLane.right_fit_m = right_fit_m
            rightLane.rightCurvature = rightCurvature
            
        return displayLanes(img, left_fit, right_fit, left_fit_m, right_fit_m, leftCurvature, rightCurvature, diffFromVehicle,M)

    clip = myclip.fl_image(processImage)
    clip.write_videofile(outputVideo, audio=False)

# Project video
videoPipeline('project_video.mp4', 'Advanced_Lane_Detection.mp4')

[MoviePy] >>>> Building video Advanced_Lane_Detection.mp4
[MoviePy] Writing video Advanced_Lane_Detection.mp4


100%|█████████▉| 1260/1261 [05:22<00:00,  4.05it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Advanced_Lane_Detection.mp4 

